In [89]:
import boto3
import os
import pandas as pd
import yfinance as yf
import json
from dotenv import load_dotenv

Configurations

In [90]:
load_dotenv("enviroment/.env")

aws_access_key = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")
aws_region = os.getenv("AWS_REGION")
bucket_name = "silver-finance-data"

In [91]:
s3_client = boto3.client(
    "s3",
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    region_name=os.getenv("AWS_REGION"),
)


Functions

In [92]:
def put_json_yfinance_aws(s3_client, ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    data = data.reset_index(drop=False) 
    for index, rows in data.iterrows():
        print(rows)
        date = rows['Date']
        close = rows['Close']
        high = rows['High']
        low = rows['High']
        open = rows['Open']
        print(type(date))
        json_data = {
            "date" : date,
            "close" : close,
            "high" : high,
            "low" : low,
            "open" : open
        }
        print(type(json_data))


        # s3_client.put_object(
        #     Bucket='silver-finance-data',
        #     Key=(key),
        #     Body=json_data,
        #     ContentType="application/json"
    return 1

In [93]:
ticker = '^BVSP'
start_date = '1900-01-01'
end_date = '2024-09-01'
data = put_json_yfinance_aws(s3_client,ticker, start_date, end_date)

[*********************100%***********************]  1 of 1 completed

Price   Ticker
Date              1993-04-27 00:00:00
Close   ^BVSP                    24.5
High    ^BVSP                    25.4
Low     ^BVSP                    24.5
Open    ^BVSP               24.799999
Volume  ^BVSP                       0
Name: 0, dtype: object
<class 'pandas.core.series.Series'>
<class 'dict'>
Price   Ticker
Date              1993-04-28 00:00:00
Close   ^BVSP               24.299999
High    ^BVSP                    24.6
Low     ^BVSP               23.700001
Open    ^BVSP                    24.5
Volume  ^BVSP                       0
Name: 1, dtype: object
<class 'pandas.core.series.Series'>
<class 'dict'>
Price   Ticker
Date              1993-04-29 00:00:00
Close   ^BVSP               23.700001
High    ^BVSP               24.799999
Low     ^BVSP               23.700001
Open    ^BVSP               24.299999
Volume  ^BVSP                       0
Name: 2, dtype: object
<class 'pandas.core.series.Series'>
<class 'dict'>
Price   Ticker
Date              1993-04-30 00:00

KeyboardInterrupt: 